In [13]:
import os
import requests
import googlemaps
import json
from gomaps import maps_search
import time
import googlemaps
import pandas as pd
import pgeocode
import requests
import json
import re

In [51]:
#Function to extract place_id of user's google maps link
def get_location_from_url(url):
    # Regular expression to match latitude and longitude
    regex = r"@(-?\d+\.\d+),(-?\d+\.\d+)"
    
    # Search for latitude and longitude in the URL
    match = re.search(regex, url)
    
    if match:
        # Extract latitude and longitude from the regex match
        latitude = float(match.group(1))
        longitude = float(match.group(2))
        
        return (latitude, longitude)
    else:
        return None
    

In [52]:
def get_place_id(latitude, longitude, api_key):
    # Set up the API endpoint URL and parameters
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    params = {
        "location": f"{latitude},{longitude}",
        "rankby": "distance",
        "category": "restaurant",
        "key": api_key
    }
    
    # Send the API request and get the JSON response
    response = requests.get(url, params=params).json()
    
    # Check if any results were returned
    if len(response["results"]) > 0:
        # Extract the place ID from the first result
        place_id = response["results"][0]["place_id"]
        return place_id
    else:
        print("Error in get_place_id")

In [156]:
def extract_place_details(place_id, api_key):
    url = f'https://maps.googleapis.com/maps/api/place/details/json?place_id={place_id}&key={api_key}'

    # Send a GET request to the API endpoint and retrieve the response
    response = requests.get(url)

    # Parse the JSON response and convert it to a dictionary
    result = json.loads(response.text)

    try:
        return [result["result"]]
    
    except:
        print("Error in extract_place_details")

In [54]:
def transform_to_df(place_details):
    user_input_details = pd.DataFrame(data = place_details)
    return user_input_details

In [198]:
#Option 1: get user details based on google maps link
def process_user_input_details1():
    user_input = input("please insert the goolge maps link of your favorite location: ")
    api_key = open("/Users/lukasbauerschmidt/Desktop/Ironhack/5. Final Project/2. Google API Key/key.txt").read()
    latitude, longitude = get_location_from_url(user_input)
    place_id = get_place_id(latitude, longitude, api_key)
    place_details = extract_place_details(place_id, api_key)
    user_input_details = transform_to_df(place_details)
    
    return user_input_details

In [199]:
#Option 2: get user details based on place_id searched in the following website "https://developers.google.com/maps/documentation/javascript/examples/places-placeid-finder"
def process_user_input_details2():
    print("1. Please find your place_id of your favorite location using the following link: https://developers.google.com/maps/documentation/javascript/examples/places-placeid-finder")
    place_id = input("please insert the place_id of your desired location: ")
    api_key = open("/Users/lukasbauerschmidt/Desktop/Ironhack/5. Final Project/2. Google API Key/key.txt").read()
    place_details = extract_place_details(place_id, api_key)
    user_input_details = transform_to_df(place_details)
    
    return user_input_details

In [ ]:
def main():
    df

In [200]:
def get_coordinates_from_details(user_input_details):
    latitude = round(user_input_details["geometry"][0]["location"]["lat"], 7)
    longitude = round(user_input_details["geometry"][0]["location"]["lng"], 7)
    return latitude, longitude

In [201]:
def process_user_input_high_level(user_input_details):
    latitude, longitude = get_coordinates_from_details(user_input_details)
    
    #Google Maps API parameters
    search_string = 'restaurant'
    distance = 0.0001
    rankby=distance
    response = map_client.places_nearby(location=(latitude, longitude), keyword=search_string, radius=distance)
    
    result = response.get('results')

    for i, business in enumerate(result):
        latitude_hl = business["geometry"]["location"]["lat"] 
        longitude_hl = business["geometry"]["location"]["lng"]
        
        if latitude_hl == latitude and longitude_hl == longitude:
            high_level_result = transform_to_df([result[i]])
            
            return high_level_result
        
        else:
            print("Error in process_user_input_high_level")

In [139]:
user_details = process_user_input_details()

1. Please find your place_id of your favorite location using the following link: https://developers.google.com/maps/documentation/javascript/examples/places-placeid-finder
please insert the place_id of your desired location: ChIJywdI5etRqEcRUVD-ycuGL6M


In [140]:
latitude, longitude = get_coordinates(user_details)
latitude

52.52858310000001

In [141]:
result = process_user_input_high_level(user_details, latitude, longitude)

In [155]:
result[0]["geometry"]["location"]["lng"]

13.3879606

{'business_status': 'OPERATIONAL',
 'geometry': {'location': {'lat': 52.526436, 'lng': 13.3879606},
  'viewport': {'northeast': {'lat': 52.52769662989272,
    'lng': 13.38932197989272},
   'southwest': {'lat': 52.52499697010728, 'lng': 13.38662232010728}}},
 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/restaurant-71.png',
 'icon_background_color': '#FF9E67',
 'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/restaurant_pinlet',
 'name': 'Gambrinus',
 'opening_hours': {'open_now': True},
 'photos': [{'height': 1386,
   'html_attributions': ['<a href="https://maps.google.com/maps/contrib/110227297314101146790">Michael Ellerman</a>'],
   'photo_reference': 'AfLeUgMTNKxA1vRgWGG6m_uTI8XnxMeCPNk0A88XrSgZdMWk3-TQm9_p4hz_hdGWn6OddGlKPWIqK-J--HeKYxLq7QYE9tR8W0c-Nm3TbS6rWXZwm885RhWxCUEwI1x5QbJBWMhDNzB4IF0rQdadQ4tFowq6ULQa2DDvzWI02WIp552J-fc',
   'width': 1386}],
 'place_id': 'ChIJU3egc-lRqEcRj_nx52zaxl8',
 'plus_code': {'compound_code': 'G9GQ+H5 

In [143]:
df = pd.DataFrame(data=result)
df

,business_status,geometry,icon,icon_background_color,icon_mask_base_uri,name,opening_hours,photos,place_id,plus_code,rating,reference,scope,types,user_ratings_total,vicinity,price_level
0,OPERATIONAL,"{'location': {'lat': 52.526436, 'lng': 13.3879...",https://maps.gstatic.com/mapfiles/place_api/ic...,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/ic...,Gambrinus,{'open_now': True},"[{'height': 1386, 'html_attributions': ['<a hr...",ChIJU3egc-lRqEcRj_nx52zaxl8,"{'compound_code': 'G9GQ+H5 Berlin', 'global_co...",3.2,ChIJU3egc-lRqEcRj_nx52zaxl8,GOOGLE,"[restaurant, food, point_of_interest, establis...",53,"Linienstraße 133, Berlin",NaN
1,OPERATIONAL,"{'location': {'lat': 52.5274428, 'lng': 13.386...",https://maps.gstatic.com/mapfiles/place_api/ic...,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/ic...,Due Fratelli,{'open_now': True},"[{'height': 2376, 'html_attributions': ['<a hr...",ChIJj_Vp3OtRqEcRYV0ssHwRVKM,"{'compound_code': 'G9GP+XJ Berlin', 'global_co...",4.3,ChIJj_Vp3OtRqEcRYV0ssHwRVKM,GOOGLE,"[restaurant, food, point_of_interest, establis...",937,"Chausseestraße 131A, Berlin",1.0
2,OPERATIONAL,"{'location': {'lat': 52.5291035, 'lng': 13.388...",https://maps.gstatic.com/mapfiles/place_api/ic...,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/ic...,Petrarca,{'open_now': True},"[{'height': 4032, 'html_attributions': ['<a hr...",ChIJq89pru5RqEcRUvL_LoDBKcY,"{'compound_code': 'G9HQ+J9 Berlin', 'global_co...",4.6,ChIJq89pru5RqEcRUvL_LoDBKcY,GOOGLE,"[restaurant, food, point_of_interest, establis...",299,"Novalisstraße 8, Berlin",NaN
3,OPERATIONAL,"{'location': {'lat': 52.5300558, 'lng': 13.384...",https://maps.gstatic.com/mapfiles/place_api/ic...,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/ic...,La Pausa,{'open_now': True},"[{'height': 720, 'html_attributions': ['<a hre...",ChIJqVqSWeRRqEcRTB7RwSW0FF0,"{'compound_code': 'G9JM+2P Berlin', 'global_co...",4.2,ChIJqVqSWeRRqEcRTB7RwSW0FF0,GOOGLE,"[restaurant, food, point_of_interest, establis...",923,"Chausseestraße 17, Berlin",1.0
4,OPERATIONAL,"{'location': {'lat': 52.5285831, 'lng': 13.387...",https://maps.gstatic.com/mapfiles/place_api/ic...,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/ic...,Schnitzelei Mitte,{'open_now': False},"[{'height': 3871, 'html_attributions': ['<a hr...",ChIJywdI5etRqEcRUVD-ycuGL6M,"{'compound_code': 'G9HQ+C5 Berlin', 'global_co...",4.5,ChIJywdI5etRqEcRUVD-ycuGL6M,GOOGLE,"[restaurant, food, point_of_interest, establis...",4138,Chausseestr. 8 Hinterhof erreichbar über: Chau...,2.0
5,OPERATIONAL,"{'location': {'lat': 52.5264943, 'lng': 13.387...",https://maps.gstatic.com/mapfiles/place_api/ic...,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/ic...,PHO - Noodlebar (Mitte),{'open_now': True},"[{'height': 3024, 'html_attributions': ['<a hr...",ChIJQT6Vc-lRqEcRn31gC28SamM,"{'compound_code': 'G9GQ+H4 Berlin', 'global_co...",4.5,ChIJQT6Vc-lRqEcRn31gC28SamM,GOOGLE,"[restaurant, food, point_of_interest, establis...",2110,"Linienstraße 134, Berlin",NaN
6,OPERATIONAL,"{'location': {'lat': 52.5277295, 'lng': 13.386...",https://maps.gstatic.com/mapfiles/place_api/ic...,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/ic...,Hito Falafel,{'open_now': True},"[{'height': 720, 'html_attributions': ['<a hre...",ChIJJdCKFdBRqEcRO5Uz93cVPWM,"{'compound_code': 'G9HP+3P Berlin', 'global_co...",4.8,ChIJJdCKFdBRqEcRO5Uz93cVPWM,GOOGLE,"[restaurant, food, point_of_interest, establis...",299,"Chauseestraße 4, Berlin",1.0
7,OPERATIONAL,"{'location': {'lat': 52.5276532, 'lng': 13.388...",https://maps.gstatic.com/mapfiles/place_api/ic...,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/ic...,Chelany-Mitte Restaurant,{'open_now': True},"[{'height': 720, 'html_attributions': ['<a hre...",ChIJ32s6P-lRqEcRHxZ4L_q_2tk,"{'compound_code': 'G9HQ+3C Berlin', 'global_co...",4.6,ChIJ32s6P-lRqEcRHxZ4L_q_2tk,GOOGLE,"[restaurant, food, point_of_interest, establis...",1022,"Torstraße 221, Berlin",2.0
8,OPERATIONAL,"{'location': {'lat': 52.5272492, 'lng': 13.38

In [ ]:
def transform_into_same_shape()